# villager와 special npc를 합칩니다.

## 목표

- [ ] villager 필드 정리하고 special npc 합치기
- [ ] tier, ranking 합치기
  

In [2]:
from collections import defaultdict
import itertools
import json
from pprint import pprint
import re

import numpy as np
import pandas as pd

In [3]:
chars = pd.read_excel('./datasets/villagers.xlsx')
npcs = pd.read_excel('./datasets/special npcs.xlsx')
pops = pd.read_excel('./datasets/popularity.xlsx')
trans = pd.read_csv('./datasets/koname.txt', delimiter='\t')

In [4]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      463 non-null    object
 1   en      463 non-null    object
 2   ko      463 non-null    object
dtypes: object(3)
memory usage: 5.5+ KB


생각해보니 번역 파일에 빠진게 있으니 하는김에 채워넣어야 할 것 같습니다.

In [5]:
trans.drop('id', axis=1, inplace=True)
trans.head()

,en,ko
0,Cyrus,리포
1,Copper,경찰관
2,Wilbur,로드리
3,DJ KK,DJ K.K.
4,Booker,경찰관


In [6]:
missing = {
    'Shino': '요비',
    'Sasha': '미첼',
    'Ione': '스피카',
    'Cephalobot': '기가',
    'Tiansheng': '제천',
    'Petri': '리카',
    'Quinn': '시온',
    'Marlo': '보스터',
    'Azalea': '페튜니아',
    'Faith': '마치',
    'Chabwick': '펭구',
    'WartJr': '샘',
    'Rio': '데자네',
    'Buck(Brows)': '바야시코프',
    'Renee': '뿔님이',
    'Roswell': '삐로코',
    'Zoe': '소면',
    'OHare': '산토스',
    'Crackle(Spork)': '포크',
    'Ace': '페더',
    'Frett': '샹펜',
}

en, ko = zip(*missing.items())
# print(en, ko)
amend = pd.DataFrame({'en': en, 'ko': ko})
# amend.head()

trans = pd.concat([trans, amend], axis=0)
trans.reset_index()


,index,en,ko
0,0,Cyrus,리포
1,1,Copper,경찰관
2,2,Wilbur,로드리
3,3,DJ KK,DJ K.K.
4,4,Booker,경찰관
...,...,...,...
479,16,Zoe,소면
480,17,OHare,산토스
481,18,Crackle(Spork),포크
482,19,Ace,페더


In [7]:
trans.reset_index(inplace=True)

In [8]:
trans = trans.sort_values('en').reset_index()
trans.head()

,level_0,index,en,ko
0,64,64,???,???
1,482,19,Ace,페더
2,151,151,Admiral,일섭
3,415,415,Agent S,2호
4,83,83,Agnes,아그네스


In [9]:
trans.drop(['level_0', 'index'], axis=1, inplace=True)
trans.head()

,en,ko
0,???,???
1,Ace,페더
2,Admiral,일섭
3,Agent S,2호
4,Agnes,아그네스


In [10]:
trans.to_excel('./datasets/trans.xlsx')

## 마을사람 필드 정리

뺄 필드: Default Clothing	Wallpaper	Flooring	Furniture List	Furniture Name List	Filename	Unique Entry ID


In [11]:
todrop = [
    'Default Clothing',
    'Wallpaper',
    'Flooring',
    'Furniture List',
    'Furniture Name List',
    'Filename',
    'Unique Entry ID'
]
chars.drop(todrop, axis=1, inplace=True)
chars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       391 non-null    int64 
 1   Name             391 non-null    object
 2   Icon Image       391 non-null    object
 3   Photo Image      391 non-null    object
 4   House Image      391 non-null    object
 5   Species          391 non-null    object
 6   Gender           391 non-null    object
 7   Personality      391 non-null    object
 8   Subtype          391 non-null    object
 9   Hobby            391 non-null    object
 10  Birthday         391 non-null    object
 11  Catchphrase      391 non-null    object
 12  Favorite Song    391 non-null    object
 13  Favorite Saying  391 non-null    object
 14  Style 1          391 non-null    object
 15  Style 2          391 non-null    object
 16  Color 1          391 non-null    object
 17  Color 2          391 non-null    ob

In [12]:
chars.drop('Unnamed: 0', axis=1, inplace=True)
chars.head()

,Name,Icon Image,Photo Image,House Image,Species,Gender,Personality,Subtype,Hobby,Birthday,Catchphrase,Favorite Song,Favorite Saying,Style 1,Style 2,Color 1,Color 2,Korean Name
0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,Bird,Male,Cranky,A,Nature,1/27,aye aye,Steep Hill,Only quitters give up.,Cool,Cool,Black,Blue,일섭
1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,Squirrel,Female,Peppy,B,Fitness,7/2,sidekick,Go K.K. Rider,You gotta put the pedal to the metal!,Active,Simple,Blue,Black,2호
2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,Pig,Female,Big Sister,A,Play,4/21,snuffle,K.K. House,You reap what you sow.,Simple,Elegant,Pink,White,아그네스
3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,Gorilla,Male,Lazy,B,Fitness,10/18,ayyyeee,Go K.K. Rider,Life is a buffet. Always go back for seconds.,Active,Active,Red,White,우락
4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,Alligator,Male,Lazy,B,Play,6/9,it'sa me,Forest Life,Slow and steady wins the race.,Simple,Simple,Red,Blue,알베르트


```js
const charactersMock = {
  admiral: {
    id: "admiral",
    special: false,
    name_en: "Admiral",
    name_ko: "일섭",
    image_icon: "https://acnhcdn.com/latest/NpcIcon/brd06.png",
    image_photo: "https://acnhcdn.com/latest/NpcBromide/NpcNmlBrd06.png",
    image_house:
      "https://acnhcdn.com/drivesync/render/houses/brd06_39_Admiral.png",
    species: "Bird",
    gender: "Male",
    gender_asia: "Male",
    personality: "Cranky",
    personality_subtype: "A",
    hobby: "Nature",
    birthday: "01-27",
    birthday_month: 1,
    birthday_day: 27,
    catchphrase: "aye aye",
    favorite_song: "Steep Hill",
    favorite_saying: "Only quitters give up.",
    styles: ["Cool"],
    colors: ["Black", "Blue"],
  },
  cyrus: {
    id: "cyrus",
    special: true,
    name_en: "Cyrus",
    name_ko: "리포",
    image_icon: "https://acnhcdn.com/latest/NpcIcon/alp.png",
    image_photo: "https://acnhcdn.com/latest/NpcPoster/NpcSpAlp.png",
    gender: "Male",
    gender_asia: "Male",
    birthday: "01-26",
    birthday_month: 1,
    birthday_day: 26,
  },
};
```

In [13]:
torename = {
    'Name': 'name_en',
    'Korean Name': 'name_ko',
    'Icon Image': 'image_icon',
    'Photo Image': 'image_photo',
    'House Image': 'image_house',
    'Species': 'species',
    'Gender': 'gender',
    'Gender (Asia)': 'gender_asia',
    'Personality': 'personality',
    'Subtype': 'personality_subtype',
    'Hobby': 'hobby',
    'Birthday': 'birthday',
    'Catchphrase': 'catchphrase',
    'Favorite Song': 'favorite_song',
    'Favorite Saying': 'favorite_saying',
}

chars.rename(columns=torename, inplace=True)
chars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name_en              391 non-null    object
 1   image_icon           391 non-null    object
 2   image_photo          391 non-null    object
 3   image_house          391 non-null    object
 4   species              391 non-null    object
 5   gender               391 non-null    object
 6   personality          391 non-null    object
 7   personality_subtype  391 non-null    object
 8   hobby                391 non-null    object
 9   birthday             391 non-null    object
 10  catchphrase          391 non-null    object
 11  favorite_song        391 non-null    object
 12  favorite_saying      391 non-null    object
 13  Style 1              391 non-null    object
 14  Style 2              391 non-null    object
 15  Color 1              391 non-null    object
 16  Color 2 

### 마을 사람 후처리

특정 필드를 만들어 넣습니다.

In [14]:
chars['gender_asia'] = chars['gender'].copy()
chars.head()


,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,birthday,catchphrase,favorite_song,favorite_saying,Style 1,Style 2,Color 1,Color 2,name_ko,gender_asia
0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,Bird,Male,Cranky,A,Nature,1/27,aye aye,Steep Hill,Only quitters give up.,Cool,Cool,Black,Blue,일섭,Male
1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,Squirrel,Female,Peppy,B,Fitness,7/2,sidekick,Go K.K. Rider,You gotta put the pedal to the metal!,Active,Simple,Blue,Black,2호,Female
2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,Pig,Female,Big Sister,A,Play,4/21,snuffle,K.K. House,You reap what you sow.,Simple,Elegant,Pink,White,아그네스,Female
3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,Gorilla,Male,Lazy,B,Fitness,10/18,ayyyeee,Go K.K. Rider,Life is a buffet. Always go back for seconds.,Active,Active,Red,White,우락,Male
4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,Alligator,Male,Lazy,B,Play,6/9,it'sa me,Forest Life,Slow and steady wins the race.,Simple,Simple,Red,Blue,알베르트,Male


In [15]:
chars['special'] = list(itertools.repeat(False, len(chars)))
chars['special'].head()

0    False
1    False
2    False
3    False
4    False
Name: special, dtype: bool

In [16]:
chars['styles'] = chars.apply(
    lambda row: list(set([row['Style 1'], row['Style 2']])),
    axis=1
)
chars['styles'].head()

0               [Cool]
1     [Simple, Active]
2    [Simple, Elegant]
3             [Active]
4             [Simple]
Name: styles, dtype: object

In [17]:
chars['colors'] = chars.apply(
    lambda row: list(set([row['Color 1'], row['Color 2']])),
    axis=1
)
chars['colors'].head()

0    [Black, Blue]
1    [Black, Blue]
2    [Pink, White]
3     [Red, White]
4      [Red, Blue]
Name: colors, dtype: object

In [18]:
chars.drop(['Style 1', 'Style 2', 'Color 1', 'Color 2'], axis=1, inplace=True)
chars.head()

,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,birthday,catchphrase,favorite_song,favorite_saying,name_ko,gender_asia,special,styles,colors
0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,Bird,Male,Cranky,A,Nature,1/27,aye aye,Steep Hill,Only quitters give up.,일섭,Male,False,[Cool],"[Black, Blue]"
1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,Squirrel,Female,Peppy,B,Fitness,7/2,sidekick,Go K.K. Rider,You gotta put the pedal to the metal!,2호,Female,False,"[Simple, Active]","[Black, Blue]"
2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,Pig,Female,Big Sister,A,Play,4/21,snuffle,K.K. House,You reap what you sow.,아그네스,Female,False,"[Simple, Elegant]","[Pink, White]"
3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,Gorilla,Male,Lazy,B,Fitness,10/18,ayyyeee,Go K.K. Rider,Life is a buffet. Always go back for seconds.,우락,Male,False,[Active],"[Red, White]"
4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,Alligator,Male,Lazy,B,Play,6/9,it'sa me,Forest Life,Slow and steady wins the race.,알베르트,Male,False,[Simple],"[Red, Blue]"


## special npc 후처리 및 정리


In [19]:
npcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       61 non-null     int64 
 1   Name             61 non-null     object
 2   Icon Image       22 non-null     object
 3   Photo Image      61 non-null     object
 4   Gender           61 non-null     object
 5   Gender (Asia)    61 non-null     object
 6   Version Added    27 non-null     object
 7   NPC ID           61 non-null     object
 8   Internal ID      61 non-null     int64 
 9   Birthday         61 non-null     object
 10  Unique Entry ID  61 non-null     object
 11  Korean Name      61 non-null     object
dtypes: int64(2), object(10)
memory usage: 3.4+ KB


In [20]:
npcs.drop(
    [
        'Unnamed: 0',
        'Version Added',
        'NPC ID',
        'Internal ID',
        'Unique Entry ID'
    ],
    axis=1,
    inplace=True
)
npcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           61 non-null     object
 1   Icon Image     22 non-null     object
 2   Photo Image    61 non-null     object
 3   Gender         61 non-null     object
 4   Gender (Asia)  61 non-null     object
 5   Birthday       61 non-null     object
 6   Korean Name    61 non-null     object
dtypes: object(7)
memory usage: 1.7+ KB


In [21]:
npcs.rename(columns=torename, inplace=True)
npcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name_en      61 non-null     object
 1   image_icon   22 non-null     object
 2   image_photo  61 non-null     object
 3   gender       61 non-null     object
 4   gender_asia  61 non-null     object
 5   birthday     61 non-null     object
 6   name_ko      61 non-null     object
dtypes: object(7)
memory usage: 1.7+ KB


필요한 칼럼은 `special` 하나 뿐인 것 같습니다.


In [22]:
npcs = npcs.assign(special=lambda v: True)

이제 `chars`와 합칩니다.


In [23]:
chars = pd.concat([chars, npcs], axis=0).reset_index()
chars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                452 non-null    int64 
 1   name_en              452 non-null    object
 2   image_icon           413 non-null    object
 3   image_photo          452 non-null    object
 4   image_house          391 non-null    object
 5   species              391 non-null    object
 6   gender               452 non-null    object
 7   personality          391 non-null    object
 8   personality_subtype  391 non-null    object
 9   hobby                391 non-null    object
 10  birthday             452 non-null    object
 11  catchphrase          391 non-null    object
 12  favorite_song        391 non-null    object
 13  favorite_saying      391 non-null    object
 14  name_ko              452 non-null    object
 15  gender_asia          452 non-null    object
 16  special 

## `birthday` 숫자 필드 만들기

`birthday_month`, `birthday_day`


In [24]:
# birthday 칼럼을 mm-dd로 고칩니다.
def reformat_birthday(bdstr):
    m, d = map(int, bdstr.split('/'))
    return f'{m:02}-{d:02}'

# reformat_birthday('1/26')
chars['birthday'] = chars['birthday'].apply(reformat_birthday)
chars['birthday'].head()

0    01-27
1    07-02
2    04-21
3    10-18
4    06-09
Name: birthday, dtype: object

In [25]:
chars['birthday_month'] = chars['birthday'].apply(lambda v: int(v[:2]))
chars['birthday_day'] = chars['birthday'].apply(lambda v: int(v[-2:]))
chars.drop('index', axis=1, inplace=True)


In [26]:
chars[chars['name_en'].duplicated()]

,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,birthday,catchphrase,favorite_song,favorite_saying,name_ko,gender_asia,special,styles,colors,birthday_month,birthday_day


Wisp 이란 녀석이 중복이고 어떤 녀석들은 아이콘 이미지가 없습니다.

~~이건 수작업으로 해야 할 것 같기 때문에 일단 익스포트합니다.~~


In [27]:
chars.drop(409, inplace=True)

In [28]:
chars = chars.sort_values('name_en').reset_index()

## 번역

영어 속성을 번역합니다.


In [29]:
chars['hobby'].unique()

array(['Nature', 'Fitness', 'Play', 'Education', 'Fashion', 'Music', nan],
      dtype=object)

In [30]:
# styles
allstyles = set()
for stylelist in chars['styles'].dropna():
    # print(stylelist)
    for style in stylelist:
        allstyles.add(style)
    
allstyles

{'Active', 'Cool', 'Cute', 'Elegant', 'Gorgeous', 'Simple'}

In [43]:
# colors

set(color for colorlist in chars['colors'].dropna() for color in colorlist)
# len(_)

{'Beige',
 'Black',
 'Blue',
 'Brown',
 'Colorful',
 'Gray',
 'Green',
 'Light blue',
 'Orange',
 'Pink',
 'Purple',
 'Red',
 'White',
 'Yellow'}

색깔이 저거밖에 없다고?


In [44]:
villagers = pd.read_excel('./datasets/villagers.xlsx')


In [45]:
print(villagers['Color 1'].unique())
print(villagers['Color 2'].unique())
print(
    len(
        set(villagers['Color 1'].unique()) | set(villagers['Color 2'].unique())
    )
)

['Black' 'Blue' 'Pink' 'Red' 'Yellow' 'Green' 'Colorful' 'Light blue'
 'Orange' 'Purple' 'Brown' 'Beige' 'White' 'Gray']
['Blue' 'Black' 'White' 'Pink' 'Brown' 'Orange' 'Purple' 'Red' 'Colorful'
 'Gray' 'Green' 'Yellow' 'Beige' 'Light blue']
14


진짜로 없네요.


In [46]:
# personality
chars['personality'].unique()

array(['Cranky', 'Peppy', 'Big Sister', 'Lazy', 'Normal', 'Snooty',
       'Jock', 'Smug', nan], dtype=object)

In [47]:
trans_hobby = dict(
    zip(
        ['Nature', 'Fitness', 'Play', 'Education', 'Fashion', 'Music'],
        ['자연', '운동', '놀이', '교육', '패션', '음악']
    )
)
trans_hobby

{'Nature': '자연',
 'Fitness': '운동',
 'Play': '놀이',
 'Education': '교육',
 'Fashion': '패션',
 'Music': '음악'}

In [48]:
trans_style = dict(
    zip(
        ['Active', 'Cool', 'Cute', 'Elegant', 'Gorgeous', 'Simple'],
        ['액티브', '쿨', '큐트', '엘레강스', '고져스', '심플']
    )
)

In [49]:
trans_personality = dict(
    zip(
        ['Cranky', 'Peppy', 'Big Sister', 'Lazy', 'Normal', 'Snooty', 'Jock', 'Smug'],
        # 느끼, 운동광, 성숙함, 친절함, 아이돌, 먹보, 단순 활발, 무뚝뚝
        ['무뚝뚝', '아이돌', '단순 활발', '먹보', '친절함', '성숙함', '운동광', '느끼함']
    )
)

In [50]:
trans_color = dict(
    zip(
        [
            'Beige',
            'Black',
            'Blue',
            'Brown',
            'Colorful',
            'Gray',
            'Green',
            'Light blue',
            'Orange',
            'Pink',
            'Purple',
            'Red',
            'White',
            'Yellow'
        ],
        [
            '베이지색',
            '검정색',
            '파랑색',
            '갈색',
            '컬러풀색',
            '회색',
            '초록색',
            '연파랑색',
            '오렌지색',
            '핑크색',
            '보라색',
            '빨강색',
            '하양색',
            '노랑색'
        ]
    )
)

In [51]:
print(villagers['Species'].unique())

['Bird' 'Squirrel' 'Pig' 'Gorilla' 'Alligator' 'Koala' 'Eagle' 'Anteater'
 'Bull' 'Mouse' 'Cat' 'Horse' 'Hamster' 'Kangaroo' 'Wolf' 'Penguin'
 'Chicken' 'Elephant' 'Sheep' 'Deer' 'Tiger' 'Cub' 'Dog' 'Bear' 'Hippo'
 'Duck' 'Goat' 'Ostrich' 'Rabbit' 'Lion' 'Frog' 'Monkey' 'Rhino' 'Octopus'
 'Cow']


In [52]:
trans_spcies = dict(
    zip(
        [
            'Bird',
            'Squirrel',
            'Pig',
            'Gorilla',
            'Alligator',
            'Koala',
            'Eagle',
            'Anteater',
            'Bull',
            'Mouse',
            'Cat',
            'Horse',
            'Hamster',
            'Kangaroo',
            'Wolf',
            'Penguin',
            'Chicken',
            'Elephant',
            'Sheep',
            'Deer',
            'Tiger',
            'Cub',
            'Dog',
            'Bear',
            'Hippo',
            'Duck',
            'Goat',
            'Ostrich',
            'Rabbit',
            'Lion',
            'Frog',
            'Monkey',
            'Rhino',
            'Octopus',
            'Cow'
        ],
        [
            '새',
            '다람쥐',
            '돼지',
            '고릴라',
            '악어',
            '코알라',
            '독수리',
            '개미핥기',
            '수소',
            '쥐',
            '고양이',
            '말',
            '햄스터',
            '캥거루',
            '늑대',
            '펭귄',
            '닭',
            '코끼리',
            '양',
            '사슴',
            '호랑이',
            '꼬마곰',
            '개',
            '곰',
            '하마',
            '오리',
            '염소',
            '타조',
            '토끼',
            '사자',
            '개구리',
            '원숭이',
            '코뿔소',
            '문어',
            '암소'
        ]
    )
)

In [53]:
def translate(ref, val):
    return ref.get(val)


In [54]:
chars['hobby'] = chars['hobby'].apply(lambda v: translate(trans_hobby, v))

In [55]:
chars['personality'] = chars['personality'].apply(
    lambda v: translate(trans_personality, v)
)

In [58]:
chars['species'] = chars['species'].apply(
    lambda v: translate(trans_spcies, v)
)

In [59]:
# chars['colors'].tail()
chars['colors'] = chars['colors'].apply(
    lambda colors: (
        [translate(trans_color, c) for c in colors] if isinstance(colors, list) else np.nan
    )
)

In [60]:
chars['styles'] = chars['styles'].apply(
    lambda styles: (
        [translate(trans_style, s) for s in styles] if isinstance(styles, list) else np.nan
    )
)

## 인기 순위 합치기

팀원들이 정발 번역명을 찾아주는 동안 인기순위를 합쳐 보겠습니다.


In [61]:
pops.drop('Unnamed: 0', axis=1, inplace=True)
pops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tier         413 non-null    int64 
 1   rank         413 non-null    int64 
 2   Korean Name  413 non-null    object
 3   Name         413 non-null    object
dtypes: int64(2), object(2)
memory usage: 9.7+ KB


In [62]:
trans_pops_tier = dict(zip(pops['Name'], pops['tier']))
trans_pops_rank = dict(zip(pops['Name'], pops['rank']))


In [63]:
chars['tier'] = chars['name_en'].apply(lambda n: trans_pops_tier.get(n))
chars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                451 non-null    int64  
 1   name_en              451 non-null    object 
 2   image_icon           412 non-null    object 
 3   image_photo          451 non-null    object 
 4   image_house          391 non-null    object 
 5   species              391 non-null    object 
 6   gender               451 non-null    object 
 7   personality          391 non-null    object 
 8   personality_subtype  391 non-null    object 
 9   hobby                391 non-null    object 
 10  birthday             451 non-null    object 
 11  catchphrase          391 non-null    object 
 12  favorite_song        391 non-null    object 
 13  favorite_saying      391 non-null    object 
 14  name_ko              451 non-null    object 
 15  gender_asia          451 non-null    obj

In [64]:
chars['rank'] = chars['name_en'].apply(lambda n: trans_pops_rank.get(n))
chars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                451 non-null    int64  
 1   name_en              451 non-null    object 
 2   image_icon           412 non-null    object 
 3   image_photo          451 non-null    object 
 4   image_house          391 non-null    object 
 5   species              391 non-null    object 
 6   gender               451 non-null    object 
 7   personality          391 non-null    object 
 8   personality_subtype  391 non-null    object 
 9   hobby                391 non-null    object 
 10  birthday             451 non-null    object 
 11  catchphrase          391 non-null    object 
 12  favorite_song        391 non-null    object 
 13  favorite_saying      391 non-null    object 
 14  name_ko              451 non-null    object 
 15  gender_asia          451 non-null    obj

티어와 랭크는 약 60개가 비어 있는 것 같습니다.


In [65]:
chars['tier'] = chars['tier'].astype(dtype=pd.Int64Dtype())
chars['rank'] = chars['rank'].astype(dtype=pd.Int64Dtype())
chars.head()

,index,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,...,favorite_saying,name_ko,gender_asia,special,styles,colors,birthday_month,birthday_day,tier,rank
0,0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,새,Male,무뚝뚝,A,자연,...,Only quitters give up.,일섭,Male,False,[None],"[None, None]",1,27,6,89
1,1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,다람쥐,Female,아이돌,B,운동,...,You gotta put the pedal to the metal!,2호,Female,False,"[None, None]","[None, None]",7,2,6,45
2,2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,돼지,Female,단순 활발,A,놀이,...,You reap what you sow.,아그네스,Female,False,"[None, None]","[None, None]",4,21,4,26
3,3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,고릴라,Male,먹보,B,운동,...,Life is a buffet. Always go back for seconds.,우락,Male,False,[None],"[None, None]",10,18,6,19
4,4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,악어,Male,먹보,B,놀이,...,Slow and steady wins the race.,알베르트,Male,False,[None],"[None, None]",6,9,6,6


## `id` 필드 만들기



In [66]:
chars.head()

,index,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,...,favorite_saying,name_ko,gender_asia,special,styles,colors,birthday_month,birthday_day,tier,rank
0,0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,새,Male,무뚝뚝,A,자연,...,Only quitters give up.,일섭,Male,False,[None],"[None, None]",1,27,6,89
1,1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,다람쥐,Female,아이돌,B,운동,...,You gotta put the pedal to the metal!,2호,Female,False,"[None, None]","[None, None]",7,2,6,45
2,2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,돼지,Female,단순 활발,A,놀이,...,You reap what you sow.,아그네스,Female,False,"[None, None]","[None, None]",4,21,4,26
3,3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,고릴라,Male,먹보,B,운동,...,Life is a buffet. Always go back for seconds.,우락,Male,False,[None],"[None, None]",10,18,6,19
4,4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,악어,Male,먹보,B,놀이,...,Slow and steady wins the race.,알베르트,Male,False,[None],"[None, None]",6,9,6,6


`ascii`같지 않은 행을 찾아봐야 합니다.


In [67]:
''.join(set(c for name in chars['name_en'] for c in name))

"hy'akDBtoGHPJAlEz.KVjYréfvCewZucWiOnbRUx dMpQq-mgSLTsIFN"

In [68]:
_toreplace = {'é': 'e', "'": '', '-': '', '.': '', ' ': ''}

`id` 규칙

- 공백은 제거
- "'"와 "-"도 제거
- '.'도 제거
- 전부 소문자
  

In [69]:
p_remove = re.compile(r"['-. ]")
chars['id'] = chars['name_en'].apply(
    lambda s: p_remove.sub('', s).lower()
)


In [70]:
p_repl = re.compile(r'é')
chars['id'] = chars['id'] = chars['id'].apply(
    lambda s: p_repl.sub('e', s).lower()
)
chars['id'].head()

0    admiral
1     agents
2      agnes
3         al
4    alfonso
Name: id, dtype: object

In [71]:
''.join(set(c for name in chars['id'] for c in name))

'hyaktolzjrfvewucinbxdpqmgs'

## 컬럼 순서 예쁘게 바꾸기

별 의미는 없습니다.


In [72]:
list(chars.columns)

['index',
 'name_en',
 'image_icon',
 'image_photo',
 'image_house',
 'species',
 'gender',
 'personality',
 'personality_subtype',
 'hobby',
 'birthday',
 'catchphrase',
 'favorite_song',
 'favorite_saying',
 'name_ko',
 'gender_asia',
 'special',
 'styles',
 'colors',
 'birthday_month',
 'birthday_day',
 'tier',
 'rank',
 'id']

In [73]:
chars.drop('index', axis=1, inplace=True)

In [74]:
chars = chars[
    [
        'id',
        'special',
        'name_en',
        'name_ko',
        'image_icon',
        'image_photo',
        'image_house',
        'species',
        'gender',
        'gender_asia',
        'birthday',
        'birthday_month',
        'birthday_day',
        'personality',
        'personality_subtype',
        'hobby',
        'catchphrase',
        'favorite_song',
        'favorite_saying',
        'styles',
        'colors',
        'tier',
        'rank',
    ]
]

## `json`으로 뽑습니다.

하지만 만약을 위해 엑셀로도 뽑아 놓습니다.


In [76]:
chars.drop('index', axis=1, inplace=True)


KeyError: "['index'] not found in axis"

In [77]:
chars.to_excel('./datasets/characters.xlsx', freeze_panes=(1, 2))

In [78]:
chars.head()

,id,special,name_en,name_ko,image_icon,image_photo,image_house,species,gender,gender_asia,...,personality,personality_subtype,hobby,catchphrase,favorite_song,favorite_saying,styles,colors,tier,rank
0,admiral,False,Admiral,일섭,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,새,Male,Male,...,무뚝뚝,A,자연,aye aye,Steep Hill,Only quitters give up.,[None],"[None, None]",6,89
1,agents,False,Agent S,2호,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,다람쥐,Female,Female,...,아이돌,B,운동,sidekick,Go K.K. Rider,You gotta put the pedal to the metal!,"[None, None]","[None, None]",6,45
2,agnes,False,Agnes,아그네스,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,돼지,Female,Female,...,단순 활발,A,놀이,snuffle,K.K. House,You reap what you sow.,"[None, None]","[None, None]",4,26
3,al,False,Al,우락,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,고릴라,Male,Male,...,먹보,B,운동,ayyyeee,Go K.K. Rider,Life is a buffet. Always go back for seconds.,[None],"[None, None]",6,19
4,alfonso,False,Alfonso,알베르트,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,악어,Male,Male,...,먹보,B,놀이,it'sa me,Forest Life,Slow and steady wins the race.,[None],"[None, None]",6,6


In [79]:
# print(chars.to_json(orient='records', indent=4))
# 빈 곳을 자꾸 null로 채우려고 해서 별로 쓸모가 없습니다.

# json.dumps(
#     {row.id: row.dropna().to_dict() for index, row in chars.iterrows()},
#     indent=4
# )
# print(_)

with open('./datasets/characters.json', 'w', encoding='utf-8') as j_out:
    json.dump(
        {row.id: row.dropna().to_dict() for index, row in chars.iterrows()},
        j_out,
        # indent=4
    )


### `json` 검증

제대로 됐나 봅니다.


In [80]:
with open('./datasets/characters.json', 'r', encoding='utf-8') as j_in:
    test = json.load(j_in)

pprint(test)

{'admiral': {'birthday': '01-27',
             'birthday_day': 27,
             'birthday_month': 1,
             'catchphrase': 'aye aye',
             'colors': [None, None],
             'favorite_saying': 'Only quitters give up.',
             'favorite_song': 'Steep Hill',
             'gender': 'Male',
             'gender_asia': 'Male',
             'hobby': '자연',
             'id': 'admiral',
             'image_house': 'https://acnhcdn.com/drivesync/render/houses/brd06_39_Admiral.png',
             'image_icon': 'https://acnhcdn.com/latest/NpcIcon/brd06.png',
             'image_photo': 'https://acnhcdn.com/latest/NpcBromide/NpcNmlBrd06.png',
             'name_en': 'Admiral',
             'name_ko': '일섭',
             'personality': '무뚝뚝',
             'personality_subtype': 'A',
             'rank': 89,
             'special': False,
             'species': '새',
             'styles': [None],
             'tier': 6},
 'agents': {'birthday': '07-02',
            'birthday_day